In [25]:
from ipynb.fs.full.shared_imports import *
import yaml

In [29]:
class MyNeuralNetwork(nn.Module):

    def __init__(self, args, output_size, device):

        super().__init__() # initialize super class
        self.device = device

        self.activation_functions = {
            'relu': nn.ReLU(), 
            'elu': nn.ELU(), 
            'tanh': nn.Tanh(),
            'softmax': nn.Softmax(dim=1),
            'softplus': nn.Softplus(),
            'sigmoid': nn.Sigmoid(),
            }
        
        self.net = self.create_sequential_net(args, output_size)
        if args['initial_bias']:
            # position of last linear layer depends on whether there is an output layer activation function
            pos = -2 if args['output_layer_activation'] else -1
            self.initialize_bias(pos, args['initial_bias'])
    
    def forward(self, observation):
        raise NotImplementedError
    
    def create_sequential_net(self, args, output_size):

        inner_layer_activations = args['inner_layer_activations']
        output_layer_activation = args['output_layer_activation']
        neurons_per_hidden_layer = args['neurons_per_hidden_layer']
        
        # define layers
        self.layers = []
        for i, output_neurons in enumerate(neurons_per_hidden_layer):
            self.layers.append(nn.LazyLinear(output_neurons))
            self.layers.append(self.activation_functions[inner_layer_activations])

        if len(neurons_per_hidden_layer) == 0:
            self.layers.append(nn.LazyLinear(output_size))
        # if there is at least one inner layer, then we know the last layers shape
        # we therefore create a Linear layer in case we want to initialize it to a certain value (not possible with LazyLinear)
        else: 
            self.layers.append(nn.Linear(neurons_per_hidden_layer[-1], output_size))
        if output_layer_activation is not None:
            self.layers.append(self.activation_functions[output_layer_activation])

        # define network as a sequence of layers
        return nn.Sequential(*self.layers)

    def initialize_bias(self, pos, value):
        self.layers[pos].bias.data.fill_(value)

class FullyConnectedNN(MyNeuralNetwork):
    """
    Fully connected neural network
    """
    
    def forward(self, observation):
        """
        Forward pass
        """
        x = observation['store_inventories']
        # inv_pos = x.sum(dim=2)
        # flatten input, except for the batch dimension
        # x = x.view(x.size(0), -1)
        x = x.flatten(start_dim=1)
        # add 1.0 to the output of the network to ensure that the output is positive at initialization
        # otherwise, when applying the clip operator (equivalently, the positive part operator), we won't 'get a gradient'
        x = self.net(x)

        # return {'stores': torch.clip(-inv_pos + x[0, 0], min=0)} # clip output to be non-negative
        return {'stores': x} # clip output to be non-negative
        # return {'stores': torch.clip(x, min=0)} # clip output to be non-negative

class BaseStock(MyNeuralNetwork):

    def forward(self, observation):
        """
        Forward pass
        """
        x = observation['store_inventories']
        inv_pos = x.sum(dim=2)
        x = x.flatten(start_dim=1)
        x = self.net(torch.tensor([0.0]).to(self.device))  # constant base stock level
        return {'stores': torch.clip(x - inv_pos, min=0)} # clip output to be non-negative



In [ ]:
def get_architecture(name):

    architectures = {
        'fully_connected': FullyConnectedNN, 
        'base_stock': BaseStock
        }
    return architectures[name]

In [27]:
if __name__ == '__main__':
    with open('config_files/one_store_backlogged.yml', 'r') as file:
        config = yaml.safe_load(file)
    keys = 'seeds', 'shifted_seeds', 'problem_params', 'params_by_dataset', 'trainer_params', 'observation_params', 'store_params', 'warehouse_params', 'optimizer_params', 'nn_params'
    seeds, shifted_seeds, problem_params, params_by_dataset, trainer_params, observation_params, store_params, warehouse_params, optimizer_params, nn_params = [config[key] for key in keys]

In [28]:
if __name__ == '__main__':
    model = FullyConnectedNN(nn_params, output_size=problem_params['n_stores'], device='cpu')

/user/ma4177/.conda/envs/MatiasRL/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
